In [ ]:
xinstall.packages("fastDummies")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



# --- Data Extraction ---

In [ ]:
library(tidyverse)
library(lubridate)
library(stringr)
library(dplyr)
library(fastDummies)

# --- Load the Dataset ---
# Load the data from the provided GitHub URL

url <- "https://raw.githubusercontent.com/mosomo82/COMP_SCI_5530/refs/heads/main/Assignment/Assignment_2%263/Q1_Used_Cars/raw_data/train.csv"
used_cars <- read.csv(url)

print("--- Original Data Loaded ---")
glimpse(used_cars)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.1     ✔ stringr   1.6.0
✔ ggplot2   4.0.0     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.2.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


[1] "--- Original Data Loaded ---"
Rows: 5,847
Columns: 14
$ X                 <int> 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, …
$ Name              <chr> "Hyundai Creta 1.6 CRDi SX Option", "Honda Jazz V", …
$ Location          <chr> "Pune", "Chennai", "Chennai", "Coimbatore", "Jaipur"…
$ Year              <int> 2015, 2011, 2012, 2013, 2013, 2016, 2013, 2012, 2018…
$ Kilometers_Driven <int> 41000, 46000, 87000, 40670, 86999, 36000, 64430, 659…
$ Fuel_Type         <chr> "Diesel", "Petrol", "Diesel", "Diesel", "Diesel", "D…
$ Transmission      <chr> "Manual", "Manual", "Manual", "Automatic", "Manual",…
$ Owner_Type        <chr> "First", "First", "First", "Second", "First", "First…
$ Mileage           <chr> "19.67 kmpl", "13 km/kg", "20.77 kmpl", "15.2 kmpl",…
$ Engine            <chr> "1582 CC", "1199 CC", "1248 CC", "1968 CC", "1461 CC…
$ Power             <chr> "126.2 bhp", "88.7 bhp", "88.76 bhp", "140.8 bhp", "…
$ Seats             <dbl> 5, 5, 7, 5, 5, 8, 5, 5, 5, 5, 5, 5,

# --- a. Look for Missing Values, Validating and Standardizing Units for All Columns

In [ ]:
# Check to see NAs or missing values
print("--- Summary of orginal Data---")
summary(used_cars)

for (col_name in names(used_cars)) {
  na_count <- sum(is.na(used_cars[col_name]))
  if (na_count > 0) {
    cat(paste0("Column '", col_name, "' has ", na_count, " NA values.\n"))
  } else {
    cat(paste0("Column '", col_name, "' has no NA values.\n"))
  }
}

# Check unique values for certain columns
columns_to_check <- c("Mileage", "Engine", "Power", "New_Price")
excluded_columns <- c("X", "Name", columns_to_check)

cat("--- Unique values for Character (String) Columns ---\n")

for (col_name in names(used_cars)) {
  if (class(used_cars[[col_name]]) == "character" && ! col_name %in% excluded_columns) {
    cat(paste0("\nUnique values for '", col_name, "':\n"))
    print(unique(used_cars[[col_name]]))
  }
}

for (col_name in columns_to_check) {
  cat(paste0("\nUnique last 4 characters for '", col_name, "':\n"))
  print(unique(str_sub(used_cars[[col_name]], -4)))
}

# Check the column 'year' in valid range 1975 - 2024
min_allowed_year <- 1975
max_allowed_year <- 2024

invalid_years <- used_cars %>%
  filter(Year < min_allowed_year | Year > max_allowed_year)

if (nrow(invalid_years) > 0) {
    print("Found invalid years:")
    print(invalid_years %>% select(Name, Year) %>% head())
} else {
    print("All years are valid.")
}

# Identify potentially invalid Kilometers_Driven values
# A threshold of 1,00,000 km is used as an initial reasonable upper limit for a used car.
# Values significantly above this could be data entry errors.
problematic_km_driven <- used_cars %>%
  filter(Kilometers_Driven > 1000000)

if (nrow(problematic_km_driven) > 0) {
  cat(paste0("Found ", nrow(problematic_km_driven), " rows with 'Kilometers_Driven' > 1,000,000 km.\n"))

  # Given the extreme outlier of 6,500,000 Kilometers_Driven, it's highly likely a data entry error.
  # I could replace it with more realistic value by dropping the zero due to the car is a big asset
  used_cars <- used_cars %>%
    mutate(Kilometers_Driven = ifelse(Kilometers_Driven == 6500000, 650000,  Kilometers_Driven))
} else {
  cat("No extreme 'Kilometers_Driven' outliers (above 1,000,000 km) found to replace.")
  used_cars_cleaned <- used_cars # If no rows are removed, keep the original dataframe
}

# Check the columns
problematic_units_rows <- used_cars %>%
  filter(!grepl("kmpl", Mileage)|!grepl("CC",Engine)|!grepl("bhp", Power))
problematic_units_rows


X,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
<int>,<chr>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5,8.61 Lakh,4.50
26,Nissan Micra Diesel XV,Hyderabad,2012,54000,Diesel,Manual,First,16.48 km/kg,1461 CC,63.1 bhp,5,,4.25
58,Nissan X-Trail SLX AT,Hyderabad,2010,121812,Diesel,Automatic,First,9.49 km/kg,1995 CC,147.6 bhp,5,,7.75
194,Honda City 1.5 GXI,Ahmedabad,2007,60006,Petrol,Manual,First,0.0 kmpl,,,NA,,2.95
208,Maruti Swift 1.3 VXi,Kolkata,2010,42001,Petrol,Manual,First,16.1 kmpl,,,NA,,2.11
733,Maruti Swift 1.3 VXi,Chennai,2006,97800,Petrol,Manual,Third,16.1 kmpl,,,NA,,1.75
749,Land Rover Range Rover 3.0 D,Mumbai,2008,55001,Diesel,Automatic,Second,0.0 kmpl,,,NA,,26.50
1294,Honda City 1.3 DX,Delhi,2009,55005,Petrol,Manual,First,12.8 kmpl,,,NA,,3.20
1327,Maruti Swift 1.3 ZXI,Hyderabad,2015,50295,Petrol,Manual,First,16.1 kmpl,,,NA,,5.80


# ---b. Remove the units from some of the attributes and only keep the numerical values---

In [ ]:
# ---b. Remove the units from some of the attributes and only keep the numerical values---

# Define a conversion factor using the approximation 1 kg CNG ≈ 1.6 Liters of Petrol
cng_to_petrol_equivalent <- 1.6
cr_to_lakh <- 100

# Cleaning units first before imputation
used_cars_cleaned <- used_cars %>%
  mutate(
      # Standardize unit for Mileage column
      Mileage = case_when(
        # If it's kmpl, just parse the number
        str_detect(Mileage, "kmpl") ~ as.numeric(str_remove(Mileage, " kmpl")),
        # If it's km/kg, parse the number and convert it
        str_detect(Mileage, "km/kg") ~ as.numeric(str_remove(Mileage, " km/kg")) * cng_to_petrol_equivalent,
        # Handle any other cases or NAs
        TRUE ~ NA_real_
      ),
      Engine = as.numeric(str_remove(Engine, " CC")),
      Power = as.numeric(str_remove(Power, " bhp")),
      # Standardize unit for New_Price column
      New_Price = case_when(
        # If it's Lakh, just parse the number
        str_detect(New_Price, "Lakh") ~ as.numeric(str_remove(New_Price, " Lakh")),
        # # If it's cr, just parse the number and convert it
        str_detect(New_Price, "cr") ~ as.numeric(str_remove(New_Price, " cr")) * cr_to_lakh,
        # Handle any other cases or NAs
        TRUE ~ NA_real_
      )
  )

glimpse(used_cars_cleaned)
summary(used_cars_cleaned)

Warning message:
“There were 4 warnings in `mutate()`.
The first warning was:
ℹ In argument: `Mileage = case_when(...)`.
Caused by warning:
! NAs introduced by coercion
ℹ Run `dplyr::last_dplyr_warnings()` to see the 3 remaining warnings.”


Rows: 5,847
Columns: 14
$ X                 <int> 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, …
$ Name              <chr> "Hyundai Creta 1.6 CRDi SX Option", "Honda Jazz V", …
$ Location          <chr> "Pune", "Chennai", "Chennai", "Coimbatore", "Jaipur"…
$ Year              <int> 2015, 2011, 2012, 2013, 2013, 2016, 2013, 2012, 2018…
$ Kilometers_Driven <dbl> 41000, 46000, 87000, 40670, 86999, 36000, 64430, 659…
$ Fuel_Type         <chr> "Diesel", "Petrol", "Diesel", "Diesel", "Diesel", "D…
$ Transmission      <chr> "Manual", "Manual", "Manual", "Automatic", "Manual",…
$ Owner_Type        <chr> "First", "First", "First", "Second", "First", "First…
$ Mileage           <dbl> 19.670, 20.800, 20.770, 15.200, 23.080, 11.360, 20.5…
$ Engine            <dbl> 1582, 1199, 1248, 1968, 1461, 2755, 1598, 1248, 1462…
$ Power             <dbl> 126.20, 88.70, 88.76, 140.80, 63.10, 171.50, 103.60,…
$ Seats             <dbl> 5, 5, 7, 5, 5, 8, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5…
$ New_Price     

       X            Name             Location              Year     
 Min.   :   1   Length:5847        Length:5847        Min.   :1998  
 1st Qu.:1510   Class :character   Class :character   1st Qu.:2012  
 Median :3015   Mode  :character   Mode  :character   Median :2014  
 Mean   :3013                                         Mean   :2013  
 3rd Qu.:4518                                         3rd Qu.:2016  
 Max.   :6018                                         Max.   :2019  
                                                                    
 Kilometers_Driven  Fuel_Type         Transmission        Owner_Type       
 Min.   :   171    Length:5847        Length:5847        Length:5847       
 1st Qu.: 33468    Class :character   Class :character   Class :character  
 Median : 52576    Mode  :character   Mode  :character   Mode  :character  
 Mean   : 57410                                                            
 3rd Qu.: 72490                                                     

# --- DATA TRANSFORMATION

# --- a.Complete part by Imputing and Replacing NAs Values

In [ ]:
# Replace NAs values with median for Mileage, Engine, Power and the most frequent value for 'Seats'
# Calculate imputation values

# Build mode() function
get_mode <- function(v) {
  uniqv <- unique(na.omit(v)) # Get unique non-NA values)
  uniqv[which.max(tabulate(match(v, uniqv)))]
}

median_mileage <- median(used_cars_cleaned$Mileage, na.rm = TRUE)
median_engine <- median(used_cars_cleaned$Engine, na.rm = TRUE)
median_power <- median(used_cars_cleaned$Power, na.rm = TRUE)
mode_seats <- get_mode(used_cars_cleaned$Seats)
# Define global median for New_Price, to be used as final fallback
global_median_new_price <- median(used_cars_cleaned$New_Price, na.rm = TRUE)

# Perform imputation
imputed_used_cars <-used_cars_cleaned %>%
  mutate(
      Mileage = ifelse(is.na(Mileage), median_mileage, Mileage),
      Engine = ifelse(is.na(Engine), median_engine, Engine),
      Power = ifelse(is.na  (Power), median_power, Power),
      Seats = ifelse(is.na(Seats), mode_seats, Seats)
  ) %>%
  # Handle New_Price with a grouped median
  group_by(Name) %>%
  mutate(
    # Create a temporary column with the median New_Price for that car's name
    New_Price_Group_Median = median(New_Price, na.rm = TRUE)
  ) %>%
  ungroup() %>%
  mutate(
    # If New_price is NA, use the grouped median
    New_Price = ifelse(is.na(New_Price), New_Price_Group_Median, New_Price),
    # If New_Price is STILL NA because the whole group was NA, use global
    New_Price = ifelse(is.na(New_Price), global_median_new_price, New_Price)
  ) %>%
  select(-New_Price_Group_Median) %>%
  # Verify New_Price >= Price
  mutate (
    New_Price = pmax(New_Price, Price, na.rm = TRUE)
  )

glimpse(imputed_used_cars)
summary(imputed_used_cars[c('Mileage','Engine','Power','New_Price','Seats')])

Rows: 5,847
Columns: 14
$ X                 <int> 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, …
$ Name              <chr> "Hyundai Creta 1.6 CRDi SX Option", "Honda Jazz V", …
$ Location          <chr> "Pune", "Chennai", "Chennai", "Coimbatore", "Jaipur"…
$ Year              <int> 2015, 2011, 2012, 2013, 2013, 2016, 2013, 2012, 2018…
$ Kilometers_Driven <dbl> 41000, 46000, 87000, 40670, 86999, 36000, 64430, 659…
$ Fuel_Type         <chr> "Diesel", "Petrol", "Diesel", "Diesel", "Diesel", "D…
$ Transmission      <chr> "Manual", "Manual", "Manual", "Automatic", "Manual",…
$ Owner_Type        <chr> "First", "First", "First", "Second", "First", "First…
$ Mileage           <dbl> 19.670, 20.800, 20.770, 15.200, 23.080, 11.360, 20.5…
$ Engine            <dbl> 1582, 1199, 1248, 1968, 1461, 2755, 1598, 1248, 1462…
$ Power             <dbl> 126.20, 88.70, 88.76, 140.80, 63.10, 171.50, 103.60,…
$ Seats             <dbl> 5, 5, 7, 5, 5, 8, 5, 5, 5, 5, 5, 5, 5, 7, 5, 5, 5, 5…
$ New_Price     

    Mileage          Engine         Power         New_Price     
 Min.   : 0.00   Min.   :  72   Min.   : 34.2   Min.   :  3.91  
 1st Qu.:15.26   1st Qu.:1198   1st Qu.: 78.0   1st Qu.: 11.66  
 Median :18.19   Median :1497   Median : 98.6   Median : 11.66  
 Mean   :18.16   Mean   :1631   Mean   :113.7   Mean   : 15.45  
 3rd Qu.:21.10   3rd Qu.:1991   3rd Qu.:138.1   3rd Qu.: 11.77  
 Max.   :28.40   Max.   :5998   Max.   :560.0   Max.   :160.00  
     Seats       
 Min.   : 2.000  
 1st Qu.: 5.000  
 Median : 5.000  
 Mean   : 5.285  
 3rd Qu.: 5.000  
 Max.   :10.000  

# --- d. More Feature for Dataset

In [15]:
# --- d. Create more feature and add this column to the dataset
engineer_used_cars <- imputed_used_cars %>%
  mutate(
    # Current age of the car
    Car_Age = year(now()) - Year,

    # Create Brand by extracting from Name
    Brand = str_split_fixed(Name, " ", 2)[, 1],

    # Kilometers per Year
    Kilometers_Per_Year = round(Kilometers_Driven / (Car_Age + 1),0),

    # Simplify Owner_Type to binary
    Is_First_Owner = ifelse(Owner_Type == "First", 1, 0)
  )

  print("--- Data with New Engineered Features (Task d) ---")
  print(head(select(
    engineer_used_cars,
    Name, Brand, Year, Car_Age, Kilometers_Per_Year, Owner_Type, Is_First_Owner
  )))

[1] "--- Data with New Engineered Features (Task d) ---"
# A tibble: 6 × 7
  Name         Brand  Year Car_Age Kilometers_Per_Year Owner_Type Is_First_Owner
  <chr>        <chr> <int>   <dbl>               <dbl> <chr>               <dbl>
1 Hyundai Cre… Hyun…  2015      10                3727 First                   1
2 Honda Jazz V Honda  2011      14                3067 First                   1
3 Maruti Erti… Maru…  2012      13                6214 First                   1
4 Audi A4 New… Audi   2013      12                3128 Second                  0
5 Nissan Micr… Niss…  2013      12                6692 First                   1
6 Toyota Inno… Toyo…  2016       9                3600 First                   1


# --- c. Transform Categorical Variables into One-Hot Encoded Value

In [16]:
encoded_used_cars <- engineer_used_cars %>%
  dummy_cols(
    select_columns = c("Fuel_Type", "Transmission", "Brand", "Location"),
    remove_selected_columns = FALSE,
    ignore_na = FALSE # NA is already handled, but for good practice
  )
print("--- Data After One-Hot Encoding (Task c) ---")
print(head(encoded_used_cars))
glimpse(encoded_used_cars)

write.csv(encoded_used_cars, "clean_used_cars.csv", row.names = FALSE)

print("Dataframes successfully saved to clean_used_cars.csv")


[1] "--- Data After One-Hot Encoding (Task c) ---"
# A tibble: 6 × 64
      X Name  Location  Year Kilometers_Driven Fuel_Type Transmission Owner_Type
  <int> <chr> <chr>    <int>             <dbl> <chr>     <chr>        <chr>     
1     1 Hyun… Pune      2015             41000 Diesel    Manual       First     
2     2 Hond… Chennai   2011             46000 Petrol    Manual       First     
3     3 Maru… Chennai   2012             87000 Diesel    Manual       First     
4     4 Audi… Coimbat…  2013             40670 Diesel    Automatic    Second    
5     6 Niss… Jaipur    2013             86999 Diesel    Manual       First     
6     7 Toyo… Mumbai    2016             36000 Diesel    Automatic    First     
# ℹ 56 more variables: Mileage <dbl>, Engine <dbl>, Power <dbl>, Seats <dbl>,
#   New_Price <dbl>, Price <dbl>, Car_Age <dbl>, Brand <chr>,
#   Kilometers_Per_Year <dbl>, Is_First_Owner <dbl>, Fuel_Type_Diesel <int>,
#   Fuel_Type_Electric <int>, Fuel_Type_Petrol <int>,
#   Transmi

# --- e. DEMONSTRATE dplyr VERBS ---

In [ ]:
# Perform select, filter, rename, mutate, arrange and summarize with group by operations

# Analysis Goal: Find the average price and Kilometers_Per_Year for 1st-owner cars, grouped by Brand
# We only look at major brands (those with > 50 cars in the dataset) and sort
# by the major brands and highest average price

print("--- Demonstrating dplyr Verbs ---")

analysis_goal <- encoded_used_cars %>%
    select(
      Brand, Location, Price, Power, Kilometers_Per_Year, Is_First_Owner
    ) %>%
    filter(Is_First_Owner == 1) %>%
    # Rename: make column Location's name clearer
    rename(City = Location) %>%
    group_by(Brand) %>%
    summarize(
      Average_Price_Lakh = round(mean(Price, na.rm = TRUE),2),
      Average_Power_BHP = round(mean(Power, na.rm = TRUE),2),
      Average_Kilometers_Per_Year = round(mean(Kilometers_Per_Year, na.rm = TRUE),2),
      Count = n(),
      .groups = 'drop'
    ) %>%

    # Mutate: Add a new column to choose popular brand or high-count groups
    mutate(
        Popular_list = ifelse (Count > 50, "Yes", "No") # Fixed typo 'Coun' to 'Count'
    ) %>%

    # Arrange: sort by highest average price
    arrange(desc(Popular_list), desc(Average_Price_Lakh))

# You can do the same for the analysis_goal dataframe
write.csv(analysis_goal, "analysis_goal.csv", row.names = FALSE)

print("Dataframes successfully saved to analysis_goal.csv")

print("--- Final Analysis Report (Task e) ---")
print(as.data.frame(analysis_goal))

[1] "--- Demonstrating dplyr Verbs ---"
[1] "Dataframes successfully saved to analysis_goal.csv"
[1] "--- Final Analysis Report (Task e) ---"
           Brand Average_Price_Lakh Average_Power_BHP
1  Mercedes-Benz              29.06            192.66
2            BMW              27.56            211.24
3           Audi              26.21            191.74
4         Toyota              12.67            128.80
5       Mahindra               8.53            121.79
6          Skoda               8.21            128.58
7           Ford               7.99            100.33
8        Hyundai               5.83             92.36
9        Renault               5.77             85.50
10         Honda               5.73            106.12
11    Volkswagen               5.37             94.08
12        Maruti               4.88             75.33
13        Nissan               4.81             85.52
14          Tata               4.14             80.81
15     Chevrolet               3.19             